In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
url = 'https://www.mycarforum.com/forums/'
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

In [6]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]
a = [a_ for a_ in a if '/forums/forum/' in a_]
forums = list(set(a))

In [7]:
forums

['https://www.mycarforum.com/forums/forum/96-non-car-related/',
 'https://www.mycarforum.com/forums/forum/67-malaysian-talk/',
 'https://www.mycarforum.com/forums/forum/15-lite-amp-ez/',
 'https://www.mycarforum.com/forums/forum/107-faq-amp-help/',
 'https://www.mycarforum.com/forums/forum/3-conti-talk/',
 'https://www.mycarforum.com/forums/forum/129-new-age-polish/',
 'https://www.mycarforum.com/forums/forum/89-car-brands/',
 'https://www.mycarforum.com/forums/forum/127-investment-amp-financial-matters/',
 'https://www.mycarforum.com/forums/forum/137-electric-cars/',
 'https://www.mycarforum.com/forums/forum/126-property-buzz/',
 'https://www.mycarforum.com/forums/forum/5-cosmetics/',
 'https://www.mycarforum.com/forums/forum/110-football-channel/',
 'https://www.mycarforum.com/forums/forum/16-korean-talk/',
 'https://www.mycarforum.com/forums/forum/49-motorsports/',
 'https://www.mycarforum.com/forums/forum/60-tips-and-resources/',
 'https://www.mycarforum.com/forums/forum/93-afterma

In [35]:
def get_href(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    try:
        max_page = [a_.text for a_ in a if 'Page 1 of ' in a_.text][0]
        max_page = int(max_page.split('Page 1 of ')[-1].split()[-1])
    except:
        max_page = -1
    
    a = [a_.get('href') for a_ in a]
    a = [a_ for a_ in a if a_ and 'forums/topic/' in a_]
    a = ['/'.join(a_.split('/')[:6]) for a_ in a]
        
    return a, max_page

In [38]:
get_href(forums[1])

(['https://www.mycarforum.com/forums/topic/2716698-itzjuzcarbon-can-customise-any-of-your-steering-wheels-car-parts-interior-trims-side-panels-and-accessories-with-carbon-fiber',
  'https://www.mycarforum.com/forums/topic/2716698-itzjuzcarbon-can-customise-any-of-your-steering-wheels-car-parts-interior-trims-side-panels-and-accessories-with-carbon-fiber',
  'https://www.mycarforum.com/forums/topic/2716698-itzjuzcarbon-can-customise-any-of-your-steering-wheels-car-parts-interior-trims-side-panels-and-accessories-with-carbon-fiber',
  'https://www.mycarforum.com/forums/topic/2716698-itzjuzcarbon-can-customise-any-of-your-steering-wheels-car-parts-interior-trims-side-panels-and-accessories-with-carbon-fiber',
  'https://www.mycarforum.com/forums/topic/2716698-itzjuzcarbon-can-customise-any-of-your-steering-wheels-car-parts-interior-trims-side-panels-and-accessories-with-carbon-fiber',
  'https://www.mycarforum.com/forums/topic/2716698-itzjuzcarbon-can-customise-any-of-your-steering-wheels

In [41]:
forums[1]

'https://www.mycarforum.com/forums/forum/67-malaysian-talk/'

In [40]:
!rm -rf forums
!mkdir forums

In [44]:
max_worker = 1

forums = sorted(forums)
for i in tqdm(range(len(forums))):
    filename = os.path.join('forums', f'{i}.json')
    
    if os.path.exists(filename) and os.path.getsize(filename) > 1000:
        continue
        
    url = forums[i]
    filtered, max_page = get_href(url)
    for i in tqdm(range(2, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}page/{a}' for a in aranged]
        
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_href, url): url for url in urls}

            for future in as_completed(futures):
                filtered.extend(future.result()[0])
                
    with open(filename, 'w') as fopen:
        json.dump(filtered, fopen)

  6%|██▍                                         | 2/36 [00:04<01:22,  2.42s/it]
0it [00:00, ?it/s]
 36%|███████████████▌                           | 13/36 [00:12<00:20,  1.10it/s]


 14%|█████▊                                    | 17/124 [00:16<01:52,  1.05s/it]


 92%|█████████████████████████████████████▋   | 114/124 [01:48<00:07,  1.36it/s]


  5%|██▏                                      | 85/1599 [01:20<22:53,  1.10it/s]


 11%|████▌                                   | 182/1599 [03:08<26:21,  1.12s/it]


...

 71%|██████████████████████████████▍            | 51/72 [00:46<00:19,  1.10it/s]


 29%|████████████▏                             | 74/256 [01:06<02:50,  1.07it/s]


 67%|███████████████████████████▍             | 171/256 [02:40<01:18,  1.08it/s]


  4%|█▌                                          | 2/55 [00:01<00:44,  1.19it/s]


 83%|██████████████████████████████████▏      | 30/36 [1:09:24<07:58, 79.69s/it]
0it [00:00, ?it/s]
 89%|████████████████████████████████████▍    | 32/36 [1:09:34<02:47, 41.97s/it]
0it [00:00, ?it/s]
 92%|█████████████████████████████████████▌   | 33/36 [1:09:34<01:28, 29.50s/it]
0it [00:00, ?it/s]
 94%|██████████████████████████████████████▋  | 34/36 [1:09:35<00:41, 20.78s/it]
0it [00:00, ?it/s]
 97%|███████████████████████████████████████▊ | 35/36 [1:09:35<00:14, 14.71s/it]
0it [00:00, ?it/s]
100%|████████████████████████████████████████| 36/36 [1:09:36<00:00, 116.01s/it]


In [45]:
filtered[0]

'https://www.mycarforum.com/forums/topic/2718927-not-the-time-for-environment-tax-on-sia-during-crisis-says-transport-minister-ong-ye-kung-to-wps-jamus-lim'